In [5]:
list(map(lambda x: x * x, range(10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [ ]:
import numpy as np
import pandas as pd
import scipy.spatial as sp
import scipy.interpolate as snt
import biocircuits
import tqdm
from math import ceil

import colorcet as cc
import holoviews as hv
import bokeh.io
import bokeh.plotting
import bokeh_catplot

from datetime import date

hv.extension('matplotlib')
bokeh.io.output_notebook()

colors = cc.palette.glasbey_category10

<hr>

Below I'm testing some of the OOP code that is intended to enable arbitrary cell types and arbitrary RHS functions. It's not working...

In [198]:
%run lattice_oop.py

In [199]:
lax = csv_to_VoronoiLattice(
    "lattice_df.csv",
    R=5,
    sigma=0.15,
    csv_kwargs=dict(index_col=0),
    lattice_kwargs=dict(init_state="static"),
)

In [200]:
lax.assign_types({"sender": 1, "transceiver": -1}, method="center", center_type="sender")

In [202]:
I_t = lambda t, **kwargs: 1

def sender_rhs(E, t, lattice, params, **kwargs): 
    mu = params[0]
    return I_t(t) - mu * E

def rhs_tc_cis(E, t, lattice, params, end_time, **kwargs):
    """"""
    alpha, k_s, p_s, mu, delta = params
    
    # Get the signal input E_bar to each cell
    A = lattice.transition_mtx(t, end_time)
    E_bar = np.dot(A, E)

    # Evaluate Hill term with cis-inhibition
    f = biocircuits.reg.act_hill(E_bar / (k_s + delta * E), p_s)
    
    # Return dE_dt
    return alpha * f - mu * E

In [203]:
t = np.linspace(0, 4, 200)
rhs_dict = {'sender': sender_rhs, "transceiver": rhs_tc_cis}
E0_dict = {'sender': np.array([I_t(t[0])]), "transceiver": np.zeros((1,))}
rxn = Reaction(t, rhs_dict, E0_dict, lax)

In [204]:
alpha = 1
k_s = 0.1
p_s = 2
mu = 0.2
delta = 1
# tau = 0.7

tc_params = alpha, k_s, p_s, mu, delta

rxn.set_inducer(I_t)
rxn.set_params({"sender": (mu,), "transceiver": tc_params})

In [205]:
rxn.simulate()

In [206]:
rxn.results.shape

(200, 346, 1)

In [207]:
df = rxn.results_to_df(rxn.results)

In [208]:
df.head()

,step,time,unique ID,type,X_coord,Y_coord,expression
0,0,0.0,000:00:---------------,sender,-0.143112,-0.136371,1.0
1,0,0.0,064:00:---------------,transceiver,3.704381,-1.929697,0.0
2,0,0.0,063:00:---------------,transceiver,-3.322658,2.493578,0.0
3,0,0.0,062:00:---------------,transceiver,3.283769,2.530889,0.0
4,0,0.0,061:00:---------------,transceiver,0.428418,-4.118855,0.0


In [209]:
df.loc[df['type'] == "sender", :]

,step,time,unique ID,type,X_coord,Y_coord,expression
0,0,0.000000,000:00:---------------,sender,-0.143112,-0.136371,1.000000e+00
68,1,0.020101,000:00:---------------,sender,-0.143112,-0.136371,1.195980e+00
46,2,0.040201,000:00:---------------,sender,-0.143112,-0.136371,1.425560e+00
70,3,0.060302,000:00:---------------,sender,-0.143112,-0.136371,1.693479e+00
46,4,0.080402,000:00:---------------,sender,-0.143112,-0.136371,1.963259e+00
70,5,0.100503,000:00:---------------,sender,-0.143112,-0.136371,2.316239e+00
47,6,0.120603,000:00:---------------,sender,-0.143112,-0.136371,2.723618e+00
66,7,0.140704,000:00:---------------,sender,-0.143112,-0.136371,3.191286e+00
71,8,0.160804,000:00:---------------,sender,-0.143112,-0.136371,3.726903e+00
23,9,0.180905,000:01:--------------0,sender,0.207770,-0.146882,4.337440e+00
